# MorganFingerprint 생성 (Structure feature)

In [2]:
import pandas as pd
import numpy as np
from rdkit import Chem
from rdkit.Chem import QED
from rdkit import DataStructs
from rdkit.Chem import AllChem
from rdkit.Chem.Fingerprints import FingerprintMols
from rdkit.Chem import rdMolDescriptors

In [1]:
import os
os.getcwd()

'/data/home/ldhyun7222/hERGAT'

In [7]:
model = RandomForestClassifier(random_state=90, n_estimators=90,max_features=22, max_depth=150)

model.fit(X_train, y_train)
y_pred = model.predict(X_test)
y_pred_proba = model.predict_proba(X_test)[:,1]


pred_train = model.predict(X_train)


train_score = accuracy_score(y_train,pred_train)
Precision = precision_score(y_test, y_pred)
Recall = recall_score(y_test, y_pred)
F1_score = 2*(Precision*Recall)/(Precision+Recall)

precisions, recalls, _ = precision_recall_curve(y_test, y_pred_proba)
AUPR = auc(recalls, precisions)

print('accuracy : {0}'.format(accuracy_score(y_test, y_pred)))
print('Precision : {0}'.format(Precision))
print('Recall : {0}'.format(Recall))
print('ROC_score :', roc_auc_score(y_test, y_pred_proba))
print('AUPRC : {0}'.format(AUPR))
print("F1 score : {0}".format(F1_score))

accuracy : 0.8133669609079445
Precision : 0.8261986301369864
Recall : 0.9120982986767486
ROC_score : 0.8828254783181532
AUPRC : 0.9358414702258335
F1 score : 0.8670260557053011


In [3]:
remained_df = pd.read_csv('dataset/remained_df.csv')

In [4]:
train_mols = [Chem.MolFromSmiles(smiles)for smiles in remained_df['canonical_smiles']]

In [5]:
# mol 형태로 바뀌지 않는다면 none_list에 담기
none_list = []
for i in range(len(train_mols)):
    if train_mols[i] is None:
        none_list.append(i)
        print('none_list에 추가됨')

reg_idx = 0
for i in none_list:
    del train_mols[i - reg_idx]
    reg_idx += 1

In [6]:
# fingerprint 생성하기

bit_info_list = []
bit_info = {}
fps = []

b = 0
# mol에서 fingerprint bitvector로 변환
for a in train_mols:
    fps.append(AllChem.GetMorganFingerprintAsBitVect(a, 3, nBits = 1024, bitInfo = bit_info))
    bit_info_list.append(bit_info.copy())

In [7]:
# array 형태로 변환
arr_list = []
for i in range(len(fps)):
    array = np.zeros((0,), dtype = np.int8)
    arr_list.append(array)
    

In [8]:
for i in range(len(fps)):
    bit = fps[i]
    DataStructs.ConvertToNumpyArray(bit, arr_list[i])

In [9]:
train_x = np.stack([i.tolist() for i in arr_list])
train_finprt = pd.DataFrame(train_x)

train_finprt.head()

,0,1,2,3,4,5,6,7,8,9,...,1014,1015,1016,1017,1018,1019,1020,1021,1022,1023
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
1,0,1,1,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


# Physicochemical Properties 생성

In [10]:
import joblib
from sklearn.preprocessing import StandardScaler

In [11]:
scaler = StandardScaler()

In [12]:
train_qe = [QED.properties(mol) for mol in train_mols]
train_qe = pd.DataFrame(train_qe)
train_qe = train_qe.drop(columns = ['ROTB', 'AROM', 'ALERTS'])
train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']] = scaler.fit_transform(train_qe[['MW', 'ALOGP', 'HBA', 'HBD', 'PSA']])

In [13]:
input_df = pd.concat([train_finprt, train_qe], axis = 1)
new_remained_df = pd.concat([remained_df, input_df], axis = 1)
new_remained_df

,SMILES,Class,canonical_smiles,0,1,2,3,4,5,6,...,1019,1020,1021,1022,1023,MW,ALOGP,HBA,HBD,PSA
0,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...,1,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.177788,-0.905935,-0.240880,-0.502855
1,COc1nc2ccc(Br)cc2cc1[C@@H](c3ccccc3)[C@@](O)(C...,1,COc1nc2ccc(Br)cc2cc1[C@@H](c1ccccc1)[C@@](O)(C...,0,1,1,1,0,0,0,...,0,0,0,0,0,1.267313,2.290658,-0.412761,-0.240880,-0.812253
2,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCCc...,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.767714,-0.089251,-0.905935,-0.240880,-0.502855
3,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@@H](C3)Oc4cccc(c4...,1,Cc1ccc(CN2[C@@H]3CC[C@H]2C[C@H](Oc2cccc(C(N)=O...,0,0,0,0,0,0,0,...,1,1,0,0,0,-0.852917,-0.133940,-0.905935,-0.240880,-0.502855
4,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc4...,1,NC(=O)c1cccc(O[C@@H]2C[C@H]3CC[C@@H](C2)N3CCc2...,0,0,0,0,0,0,0,...,1,0,0,0,0,-0.917145,-0.367382,-0.905935,-0.240880,-0.502855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7925,O=C1[C@H]2N(c3ccc(OCC=CCCNCC(=O)Nc4c(Cl)cc(cc4...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCC=CCOc2ccc(c...,0,0,0,0,1,0,0,...,1,0,0,0,0,1.160303,-0.326243,0.573587,2.519498,1.530423
7926,O=C1[C@H]2N(c3ccc(OCCCCCNCC(=O)Nc4c(Cl)cc(cc4C...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCCCOc2ccc(cc...,0,0,1,0,1,0,0,...,1,0,0,0,0,1.181779,-0.166537,0.573587,2.519498,1.530423
7927,O=C1[C@H]2N(c3ccc(OCC=CCCCNCC(=O)Nc4c(Cl)cc(cc...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCC=CCOc2ccc(...,0,0,0,0,1,0,0,...,1,0,0,0,0,1.309733,-0.048112,0.573587,2.519498,1.530423
7928,O=C1[C@H]2N(c3ccc(OCCCCCCNCC(=O)Nc4c(Cl)cc(cc4...,0,NC1=NCc2cc(Cl)c(c(Cl)c2)NC(=O)CNCCCCCCOc2ccc(c...,0,0,1,0,1,0,0,...,1,0,0,0,0,1.331210,0.111594,0.573587,2.519498,1.530423


In [14]:
new_remained_df.to_csv('dataset/new_remained_df.csv', index = False)